## Direct solve

In [ ]:
using OptimalControl

# Parameters
Cd = 310.
Tmax = 3.5
β = 500.
b = 2.
N = 100
t0 = 0.
r0 = 1.
v0 = 0.
vmax = 0.1
m0 = 1.
mf = 0.6

ocp = Model()

@variable tf
@time t ∈ (t0, tf))
@state x[3]
@control u

@constraint(ocp, x(0) == [ r0, v0, m0 ], :initial_con)
@constraint(ocp, 0. ≤ u(t) ≤ 1.)
@constraint(ocp, r0 ≤ x(t)[1], :state_con1)
@constraint(ocp, 0 ≤ x(t)[2] ≤ vmax, :state_con2)
@constraint(ocp, m0 ≤ x(t)[3] ≤ mf, :state_con3)

@objective(ocp, r(tf), Max)

function F0(x)
    r, v, m = x
    D = Cd * v^2 * exp(-β*(r-1.))
    F = [ v, -D/m-1.0/r^2, 0. ]
    return F
end

function F1(x)
    r, v, m = x
    F = [ 0., Tmax/m, -b*Tmax ]
    return F
end

function f!(x, u, dx)
    dx[:] = F0(x(t)) + u(t)*F1(x(t))
end

@constraint(ocp, f!(x(t), u(t), ∂x(t)))

sol = solve(ocp)
plot(sol)

## Indirect solve

In [ ]:
# Bang controls
u0(x, p) = 0.
u1(x, p) = 1.

# Computation of singular control of order 1
H0(x, p) = p' * F0(x)
H1(x, p) = p' * F1(x)
H01 = {H0, H1}
H001 = {H0, H01}
H101 = {H1, H01}
us(x, p) = -H001(x, p) / H101(x, p)

# Computation of boundary control
remove_constraint!(ocp, :state_con1)
remove_constraint!(ocp, :state_con3)
@constraint(ocp, x(tf)[3] == mf, :final_con) # changed to equality constraint for shooting
g = vmax - constraint(ocp, :state_con2)
ub(x) = -Ad(F0, g)(x) / Ad(F1, g)(x)
μb(x, p) = H01(x, p) / Ad(F1, g)(x)

f0 = Flow(ocp, u0)
f1 = Flow(ocp, u1)
fs = Flow(ocp, us)
fb = Flow(ocp, ub, μb)

# Shooting function
function shoot!(x0, p0, t1, t2, t3, tf, s) # B+ S C B0 structure

    x1, p1 = f1(t0, x0, p0, t1)
    x2, p2 = fs(t1, x1, p1, t2)
    x3, p3 = fb(t2, x2, p2, t3)
    xf, pf = f0(t3, x3, p3, tf)
    s[1:3] = x0 - constraint(ocp, :initial_con)
    s[4] = constraint(ocp, :final_con)(t0, x0, tf, xf) - mf
    s[5:6] = pf[1:2] - [ 1.0, 0.0 ]
    s[7] = H1(x1, p1)
    s[8] = H01(x1, p1)
    s[9] = g(x2)
    s[10] = H0(xf, pf) # free tf

end

# Initialisation from direct solution
x = sol.state
p = sol.adjoint
H1_plot = plot(t, H1.(x, p), xlabel = "t", ylabel = "H1", legend = false)
g_plot = plot(t, g.(x), xlabel = "t", ylabel = "g", legend = false)
display(plot(u_plot, H1_plot, g_plot, layout = (3,1)))
η = 1e-3
t13 = t[ abs.(H1.(x, p)) .≤ η ]
t23 = t[ 0 .≤ g.(x) .≤ η ]
p0 = p[1]
t1 = min(t13...)
t2 = min(t23...)
t3 = max(t23...)
tf = t[end]
ξ = [ x0 ; p0 ; t1 ; t2 ; t3 ; tf ]

println("Initial guess:\n", ξ)

# Solve
nle = NonlinearEqu((ξ, s) -> shoot!(ξ[1:3], ξ[4:6], ξ[7], ξ[8], ξ[9], ξ[10], s))
sol = solve(nle)

# Plots
p0 = sol.val[4:6]
t1 = sol.val[7]
t2 = sol.val[8]
t3 = sol.val[9]
tf = sol.val[10]

f = f1 @ (t1, fs) @ (t2, fb) @ (t3, f0) # composition of the four Hamiltonian flows

plot(f, (t0, x0, p0, tf))